<a href="https://colab.research.google.com/github/snyamson/LP5-Twitter-Sentiment-Analysis/blob/main/LP_5_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALLATIONS

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install gradio transformers -q

# IMPORTATIONS

In [ ]:
# Import Libraries
import gradio as gr

# Data Manipulation
import numpy as np
import pandas as pd

# Serialization
import pickle

# Transformers
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# Index to words
from scipy.special import softmax

# APP BUILDING

In [ ]:
# Model Path
model_path = 'snyamson/covid-tweet-sentiment-analyzer-roberta-latest'

In [ ]:
# Load Tokenizers, Config and Model
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

### TEXT PREPROCESSING

In [ ]:
# Define function to preprocess the text data
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return " ".join(new_text)

### PREDICTION FUNCTION

In [ ]:
# Define function to encode the data and predict the sentiment
def sentiment_analysis(text):
    text = preprocess(text)

    encoded_input = tokenizer(text, return_tensors = "pt")
    output = model(**encoded_input)
    scores_ = output[0][0].detach().numpy()
    scores_ = softmax(scores_)

    # Format output dict of scores
    labels = ["Negative", "Neutral", "Positive"]
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }

    return scores

# USER INTERFACE

In [ ]:
# Define the Application User Interface
ui = gr.Interface(fn=sentiment_analysis,
                  inputs=gr.Textbox("Write your text or tweet here..."),
                  outputs="label",
                  title="Sentiment Analysis of Tweets on COVID-19 Vaccines",
                  description="To vaccinate or not? This app analyzes sentiment of text based on tweets tweets about COVID-19 Vaccines using a fine-tuned roBERTA model",
                  examples=[["The idea of a vaccine in record time sure sounds interesting!"], ["The Vaccination is dubious. Don't Try it else you will die early!"], [ "Hmmmmm! All these are works of the devil oooo"]])

In [ ]:
# Run the application
ui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ca82ef5c4ab5d84549.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Get requirements
!pip freeze > requirements.txt